In [ ]:
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#Constants
savename = ""
num_classes = 38
learning_rate = 3e-4
batch_size = 128
num_epochs = 20

In [ ]:
"""
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
source_folder = "/" #Path to the folder with the images
train_folder = "/" #Path to the folder where you want to store the train images
test_folder = "/" #Path to the folder where you want to store the test images

# Create train and test folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get a list of all subfolders in the source folder
subfolders = [f for f in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, f))]

for subfolder in subfolders:
    # Create subfolders in train and test folders
    os.makedirs(os.path.join(train_folder, subfolder), exist_ok=True)
    os.makedirs(os.path.join(test_folder, subfolder), exist_ok=True)

    # Get a list of all image files in the subfolder
    image_files = [f for f in os.listdir(os.path.join(source_folder, subfolder)) if os.path.isfile(os.path.join(source_folder, subfolder, f))]

    # Split the dataset into train and test sets
    train_files, test_files = train_test_split(image_files, test_size=0.3, random_state=42) # SPLIT SIZE CHANGE HERE

    # Move files to respective folders
    for file_name in train_files:
        source_path = os.path.join(source_folder, subfolder, file_name)
        destination_path = os.path.join(train_folder, subfolder, file_name)
        shutil.copy(source_path, destination_path)

    for file_name in test_files:
        source_path = os.path.join(source_folder, subfolder, file_name)
        destination_path = os.path.join(test_folder, subfolder, file_name)
        shutil.copy(source_path, destination_path)
"""

In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define your data directories====Non-Augmented
train_dir = "/home/user/Training"  # Location of the Training Data
val_dir = "/home/user/Validation"  # Location of the Validation Data

# Define mean and standard deviation for ImageNet
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Add normalization to your transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Create datasets
train_dataset = ImageFolder(root=train_dir, transform=train_transform)
val_dataset = ImageFolder(root=val_dir, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Check the number of images in the training dataloader
num_train_images = len(train_loader.dataset)
print(f"Number of images in the training dataloader: {num_train_images}")

# Check the number of images in the validation dataloader
num_val_images = len(val_loader.dataset)
print(f"Number of images in the validation dataloader: {num_val_images}")

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForImageClassification, AutoConfig

class CustomPM(nn.Module):
    def __init__(self, num_classes, pretrained_model="facebook/convnextv2-base-1k-224", hidden_size=256):
        super(CustomPM, self).__init__()

        config = AutoConfig.from_pretrained(pretrained_model)
        self.cvnt = AutoModelForImageClassification.from_pretrained(pretrained_model, config=config)
        
        # Freeze ConvNeXt-V2 model
        for param in self.cvnt.parameters():
            param.requires_grad = False

        # Extracting the original classifier layer
        original_classifier = self.cvnt.classifier

        self.cvnt.classifier = nn.Sequential(
            nn.Linear(original_classifier.in_features, hidden_size),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )        
    def forward(self, images):
        outputs = self.cvnt(images)
        logits = outputs.logits
        return logits

model = CustomPM(num_classes)
model.to(device)

In [ ]:
import os
from torch.optim import AdamW
from lion_pytorch import Lion
from sklearn.metrics import precision_score, recall_score, f1_score

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

#Comment out the optimizer you don't want to use

optimizer = Lion(model.parameters(), lr=learning_rate)
#optimizer = AdamW(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()  # Set model to training mode
    train_loss = 0.0
    correct_train = 0

    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
    
    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = correct_train / len(train_loader.dataset) * 100
    
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()

        # Collect predictions and labels for later evaluation
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct_val / len(val_loader.dataset) * 100

    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

    # Print the epoch statistics
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

# Save the model
final_checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
torch.save(final_checkpoint, savename + '.pt')